In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn import model_selection, preprocessing
import xgboost as xgb
import datetime

#load files
train = pd.read_csv('train.csv', parse_dates=['timestamp'])
test = pd.read_csv('test.csv', parse_dates=['timestamp'])
macro = pd.read_csv('macro.csv', parse_dates=['timestamp'])
id_test = test.id

In [23]:
print(train.shape)
print(test.shape)

(30471, 292)
(7662, 291)


In [2]:
bad_index = train[train.life_sq > train.full_sq].index
bad_index[0:5]

Int64Index([1084, 1188, 1822, 1863, 2009], dtype='int64')

In [3]:
train[1084:1085]

,id,timestamp,full_sq,life_sq,floor,max_floor,material,build_year,num_room,kitch_sq,...,cafe_count_5000_price_2500,cafe_count_5000_price_4000,cafe_count_5000_price_high,big_church_count_5000,church_count_5000,mosque_count_5000,leisure_count_5000,sport_count_5000,market_count_5000,price_doc
1084,1085,2012-02-06,44,281.0,6.0,NaN,NaN,NaN,NaN,NaN,...,9,4,0,13,22,1,0,52,4,6200000


In [4]:
type(bad_index)

pandas.indexes.numeric.Int64Index

In [5]:
bad_index.shape

(37,)

In [6]:
train.ix[bad_index, 2:4]

,full_sq,life_sq
1084,44,281.0
1188,9,44.0
1822,18,38.0
1863,30,178.0
2009,5,40.0
4385,73,426.0
6336,37,191.0
6531,80,88.0
6993,73,77.0
7208,31,195.0


In [7]:
train.ix[bad_index, "life_sq"] = np.NaN

In [8]:
train.ix[1084:1085,:]

,id,timestamp,full_sq,life_sq,floor,max_floor,material,build_year,num_room,kitch_sq,...,cafe_count_5000_price_2500,cafe_count_5000_price_4000,cafe_count_5000_price_high,big_church_count_5000,church_count_5000,mosque_count_5000,leisure_count_5000,sport_count_5000,market_count_5000,price_doc
1084,1085,2012-02-06,44,NaN,6.0,NaN,NaN,NaN,NaN,NaN,...,9,4,0,13,22,1,0,52,4,6200000
1085,1086,2012-02-06,13,9.0,3.0,NaN,NaN,NaN,NaN,NaN,...,27,10,0,7,20,0,2,84,9,3000000


In [9]:
bad_index = test[test.life_sq > test.full_sq].index
bad_index[0:5]

Int64Index([64, 119, 171, 464, 601], dtype='int64')

In [10]:
bad_index.shape

(12,)

In [11]:
test.ix[bad_index,2:4]

,full_sq,life_sq
64,29.80,30.60
119,100.04,104.28
171,57.80,60.00
464,0.00,37.80
601,74.20,742.00
1896,36.10,361.00
2027,51.90,93.20
2031,40.20,237.00
2791,86.90,869.00
2804,14.80,33.80


In [12]:
equal_index = [601,1896,2791]
test.ix[equal_index, "life_sq"] = test.ix[equal_index, "full_sq"]

In [13]:
bad_index = test[test.life_sq > test.full_sq].index

In [14]:
bad_index[0:5]

Int64Index([64, 119, 171, 464, 2027], dtype='int64')

In [15]:
bad_index.shape

(9,)

In [16]:
test.ix[bad_index, "life_sq"] = np.NaN

In [17]:
bad_index = train[train.life_sq < 5].index

In [18]:
bad_index[0:5]

Int64Index([104, 858, 1596, 2778, 3426], dtype='int64')

In [19]:
bad_index.shape

(435,)

In [21]:
train.ix[104,2:4]

full_sq    26
life_sq     1
Name: 104, dtype: object

In [22]:
train.ix[bad_index, "life_sq"] = np.NaN
bad_index = test[test.life_sq < 5].index
print(bad_index.shape)
test.ix[bad_index, "life_sq"] = np.NaN
bad_index = train[train.full_sq < 5].index
print(bad_index.shape)
train.ix[bad_index, "full_sq"] = np.NaN
bad_index = test[test.full_sq < 5].index
print(bad_index.shape)
test.ix[bad_index, "full_sq"] = np.NaN

(330,)
(26,)
(3,)


In [24]:
train.ix[13117:13118,:]

,id,timestamp,full_sq,life_sq,floor,max_floor,material,build_year,num_room,kitch_sq,...,cafe_count_5000_price_2500,cafe_count_5000_price_4000,cafe_count_5000_price_high,big_church_count_5000,church_count_5000,mosque_count_5000,leisure_count_5000,sport_count_5000,market_count_5000,price_doc
13117,13120,2013-12-14,31.0,19.0,5.0,1.0,1.0,NaN,1.0,1970.0,...,27,5,0,27,44,0,12,117,12,3650000
13118,13121,2013-12-14,39.0,23.0,6.0,12.0,5.0,1968.0,2.0,6.0,...,9,1,0,10,18,0,9,80,15,6700000


In [35]:
bad_index = train[(train.full_sq > 210) & (train.life_sq / train.full_sq < 0.3)].index

In [36]:
bad_index.shape

(7,)